In [8]:
# Célula 1: Configuração do Ambiente
import os
import pandas as pd
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from shapash import SmartExplainer

# Configuração de Caminhos (Sanitização para Windows/Web)
base_path = os.getcwd()
docs_path = os.path.join(base_path, '..', 'docs')
os.makedirs(docs_path, exist_ok=True)

# Define caminhos absolutos e troca barras invertidas por normais
output_path = os.path.abspath(os.path.join(docs_path, 'Relatorio_Auditoria_Manutencao_V1.html')).replace('\\', '/')
info_path = os.path.abspath(os.path.join(docs_path, 'project_info.yml')).replace('\\', '/')

print(f"Ambiente configurado.\nRelatório será salvo em: {output_path}")

Ambiente configurado.
Relatório será salvo em: c:/XAILabProcess/docs/Relatorio_Auditoria_Manutencao_V1.html


In [9]:
# Célula 2: Carga e Tratamento dos Dados (ETL)
# Carregamento
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00601/ai4i2020.csv"
df = pd.read_csv(url)

# Tradução e Limpeza
mapa_nomes = {
    'Type': 'Tipo_Equipamento', 'Air temperature [K]': 'Temp_Ar',
    'Process temperature [K]': 'Temp_Processo', 'Rotational speed [rpm]': 'Rotacao_rpm',
    'Torque [Nm]': 'Torque_Nm', 'Tool wear [min]': 'Desgaste_Ferramenta_min',
    'Machine failure': 'Falha'
}

# Removemos colunas identificadoras e as de falhas específicas (Data Leakage)
df = df.rename(columns=mapa_nomes).drop(['UDI', 'Product ID', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], axis=1)

print("Dados carregados e limpos.")
display(df.head())

Dados carregados e limpos.


,Tipo_Equipamento,Temp_Ar,Temp_Processo,Rotacao_rpm,Torque_Nm,Desgaste_Ferramenta_min,Falha
0,M,298.1,308.6,1551,42.8,0,0
1,L,298.2,308.7,1408,46.3,3,0
2,L,298.1,308.5,1498,49.4,5,0
3,L,298.2,308.6,1433,39.5,7,0
4,L,298.2,308.7,1408,40.0,9,0


In [10]:
# Célula 3: Preparação e Engenharia de Features
X = df.drop('Falha', axis=1)
y = df['Falha']

# Divisão Treino/Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Codificação (L, M, H -> Números)
encoder = OrdinalEncoder(cols=['Tipo_Equipamento'])
X_train_enc = encoder.fit_transform(X_train)
X_test_enc = encoder.transform(X_test)

print("Dados preparados para modelagem.")

Dados preparados para modelagem.


In [11]:
#Célula 4: Treinamento do Modelo (Random Forest)
# Usamos Random Forest para garantir compatibilidade total com o Shapash Report
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_enc, y_train)

print("Modelo Random Forest treinado com sucesso!")

Modelo Random Forest treinado com sucesso!


In [12]:
#Célula 5: Configuração do Shapash (XAI)

# Dicionário para deixar o relatório legível
features_dict = {
    'Temp_Ar': 'Temperatura do Ambiente',
    'Rotacao_rpm': 'Velocidade de Rotação',
    'Torque_Nm': 'Torque do Motor',
    'Desgaste_Ferramenta_min': 'Desgaste da Ferramenta',
    'Tipo_Equipamento': 'Tipo de Maquina',
    'Temp_Processo': 'Temperatura do Processo'
}

xpl = SmartExplainer(
    model=model,
    preprocessing=encoder,
    features_dict=features_dict
)

# Compilação (Cálculo das contribuições)
xpl.compile(x=X_test_enc, y_target=y_test)

print("Explicabilidade compilada.")

INFO: Shap explainer type - <shap.explainers._tree.TreeExplainer object at 0x00000165577AFD90>
Explicabilidade compilada.


In [13]:
#Célula 6: Criação do Arquivo de Metadados
# Conteúdo sem acentos para evitar problemas de encoding no Windows
conteudo_yaml = """
general:
  creation_date: '02/02/2026'
  description: 'Projeto PIBIC - Auditoria Industrial'
  display_logo: False
"""

with open(info_path, 'w', encoding='utf-8') as f:
    f.write(conteudo_yaml)

print(f"Arquivo de metadados criado em: {info_path}")

Arquivo de metadados criado em: c:/XAILabProcess/docs/project_info.yml


In [16]:
# Célula 7: Geração do Relatório (Bypass de Métricas)

print("Iniciando geração do relatório (Aguarde 1 a 2 minutos)...")

xpl.generate_report(
    output_file=output_path,
    project_info_file=info_path,
    x_train=X_train_enc,
    y_train=y_train,
    y_test=y_test,
    # Títulos sem acentos
    title_story="Relatorio de Auditoria - Industria 5.0",
    title_description="""
    Auditoria do modelo Random Forest para Manutencao Preditiva.
    Validacao de transparencia (XAI) e confiabilidade.
    Este documento serve como evidencia de compliance.
    """,
    # O TRUQUE: Passamos uma lista vazia.
    # Isso satisfaz a exigência "tem que ser lista" mas evita o cálculo que estava travando.
    metrics=[]
)

print("-" * 30)
print("SUCESSO FINAL! O Relatório HTML foi gerado.")
print(f"Acesse o arquivo na pasta: {output_path}")

INFO:papermill:Input Notebook:  C:\XAILabProcess\.venv\Lib\site-packages\shapash\report\base_report.ipynb
INFO:papermill:Output Notebook: C:\Users\joaor\AppData\Local\Temp\tmpl4j6c5t2\base_report.ipynb


Iniciando geração do relatório (Aguarde 1 a 2 minutos)...


Executing:  27%|██▋       | 4/15 [00:07<00:20,  1.82s/cell]


PapermillExecutionError: 
---------------------------------------------------------------------------
Exception encountered at "In [4]":
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[4], line 14
     11 y_train = load_saved_df(os.path.join(dir_path, 'y_train.csv'))
     12 y_test = load_saved_df(os.path.join(dir_path, 'y_test.csv'))
---> 14 report = ProjectReport(
     15     explainer=xpl, 
     16     project_info_file=project_info_file, 
     17     x_train=x_train, 
     18     y_train=y_train,
     19     y_test=y_test, 
     20     config=config
     21 )

File c:\XAILabProcess\.venv\lib\site-packages\shapash\report\project_report.py:123, in ProjectReport.__init__(self, explainer, project_info_file, x_train, y_train, y_test, config)
    120 print_javascript_misc()
    122 if "metrics" in self.config.keys():
--> 123     if not isinstance(self.config["metrics"], list) o